<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/dataset/corpus_compose_MultihopRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4 requests

In [ ]:
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,071 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
H

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import re
import time

# JSON 파일 로드 함수
def load_multihop_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# 웹페이지에서 텍스트를 추출하는 함수
def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 페이지에서 모든 텍스트 추출 및 정리
        page_text = soup.get_text(separator=' ', strip=True)
        clean_text = re.sub(r'\s+', ' ', page_text).strip()
        return clean_text

    except requests.RequestException as e:
        print(f"Failed to retrieve {url}: {e}")
        return ""

# 모든 URL의 텍스트를 가져와 저장하는 함수
def save_texts_from_evidence_list(json_file_path, output_file_path):
    # JSON 파일 로드
    data = load_multihop_json(json_file_path)

    # 중복된 URL 확인용 집합
    visited_urls = set()
    # 모든 evidence_list URL의 텍스트를 저장할 리스트
    all_texts = []

    # evidence_list 내 각 URL에서 텍스트 추출
    for entry in data:
        evidence_list = entry.get("evidence_list", [])
        for evidence in evidence_list:
            url = evidence.get("url")
            if url and url not in visited_urls:
                print(f"Scraping URL: {url}")
                page_text = extract_text_from_url(url)
                all_texts.append(f"{page_text}")
                # 방문한 URL 집합에 추가
                visited_urls.add(url)
                # 짧은 지연 시간 추가
                time.sleep(1)  # 서버 부하를 줄이기 위해 요청 사이에 지연 추가

    # 파일로 저장
    with open(output_file_path, 'w') as file:
        file.write("\n".join(all_texts))

    print(f"All texts from evidence_list URLs saved to {output_file_path}")

# 사용 예시
#json_file_path = '/content/drive/MyDrive/QA/multihopRAG_4type_classification_urls_answer_v2.json'  # JSON 파일 경로
#output_file_path = '/content/drive/MyDrive/QA/MultihopRAG/multihopRAG_corpus.txt'  # 출력 텍스트 파일 경로
json_file_path = '/content/MultiHopRAG_4type_classification_500_v3.json'  # JSON 파일 경로
output_file_path = '/content/multihopRAG_corpus.txt'  # 출력 텍스트 파일 경로
save_texts_from_evidence_list(json_file_path, output_file_path)


Scraping URL: https://www.theverge.com/2023/9/28/23893269/ftx-sam-bankman-fried-trial-evidence-crypto
Scraping URL: https://techcrunch.com/2023/10/01/ftx-lawsuit-timeline/
Scraping URL: https://techcrunch.com/2023/10/07/sam-altman-backs-a-teens-startup-google-unveils-the-pixel-8-and-tiktok-tests-an-ad-free-tier/
Scraping URL: https://fortune.com/2023/09/26/donald-trump-fraud-banks-insurers-real-estate-judge-new-york/
Scraping URL: https://www.theage.com.au/business/companies/the-777-million-surprise-donald-trump-is-getting-richer-20231108-p5eicf.html?ref=rss&utm_medium=rss&utm_source=rss_business
Scraping URL: https://fortune.com/2023/11/18/how-did-openai-fire-sam-altman-greg-brockman-rogue-board/
Scraping URL: https://techcrunch.com/2023/11/17/wtf-is-going-on-at-openai-sam-altman-fired/
Scraping URL: https://fortune.com/crypto/2023/10/04/sam-bankman-fried-lawyers-opening-statements-witnesses-marc-antoine-julliard-adam-yedidia-ftx-commodities-trader-developer/
Scraping URL: https://tec

In [ ]:
"""
def remove_empty_and_marker_lines(input_file, output_file):
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    # 빈 줄과 '==='로 시작하는 줄을 제외하고 새로운 리스트에 저장
    cleaned_lines = [line for line in lines if line.strip() and not line.startswith("===")]

    # 정리된 내용을 새로운 파일에 저장
    with open(output_file, 'w') as outfile:
        outfile.writelines(cleaned_lines)

    print(f"Cleaned text saved to {output_file}")

# 사용 예시
#input_file = '/content/drive/MyDrive/QA/multihopRAG_corpus.txt'  # 원본 텍스트 파일 경로
#output_file = '/content/drive/MyDrive/QA/multihopRAG_corpus_remove_url.txt'  # 정리된 텍스트 파일 경로
input_file = '/content/drive/MyDrive/QA/multihopRAG_corpus.txt'  # 원본 텍스트 파일 경로
output_file = '/content/drive/MyDrive/QA/multihopRAG_corpus_remove_url.txt'  # 정리된 텍스트 파일 경로
remove_empty_and_marker_lines(input_file, output_file)
"""

Cleaned text saved to /content/drive/MyDrive/QA/multihopRAG_corpus_remove_url.txt


In [ ]:
import requests
from bs4 import BeautifulSoup
import json

# Function to clean and extract Wikipedia content
def clean_wikipedia_content(url):
    # Send a GET request to fetch the content of the page
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Remove references, scripts, styles, and other irrelevant elements
    for element in soup(['sup', 'img', 'style', 'script']):
        element.decompose()

    # Extract the main content from bodyContent div
    content_div = soup.find('div', {'id': 'bodyContent'})
    paragraphs = content_div.find_all('p')

    # Extract the text from paragraphs and join them with spaces
    text_content = ' '.join([para.get_text(strip=True) for para in paragraphs])

    # Clean the text (remove references like [1], [2], extra whitespaces, etc.)
    clean_text = text_content.replace("\n", " ").strip()

    return clean_text

# Load the multi.json file and extract context URLs
def extract_context_texts(json_file, output_file):
    with open(json_file, 'r') as infile:
        data = json.load(infile)

    # Set to keep track of already visited URLs to avoid duplication
    visited_urls = set()

    with open(output_file, 'w') as outfile:
        total_entries = len(data)
        for index, entry in enumerate(data):
            context_urls = entry.get('context_url', [])
            for url in context_urls:
                if url not in visited_urls:
                    visited_urls.add(url)
                    try:
                        # Log the progress
                        print(f"Fetching URL {url} ({len(visited_urls)}/{total_entries})")

                        text = clean_wikipedia_content(url)
                        outfile.write(f"{text}\n")  # Save each page's text on a single line

                    except Exception as e:
                        print(f"Error fetching {url}: {e}")

# 사용 예시
#json_file = '/content/drive/MyDrive/QA/multihopRAG_4type_classification_urls_answer_v2.json'  # multi.json 파일 경로
#output_file = '/content/drive/MyDrive/QA/multihopRAG_corpus_wiki.txt'  # 추출된 텍스트를 저장할 파일 경로
json_file = '/content/MultiHopRAG_4type_classification_500_v3.json'  # multi.json 파일 경로
output_file = '/content/multihopRAG_corpus_wiki.txt'  # 추출된 텍스트를 저장할 파일 경로
extract_context_texts(json_file, output_file)


Fetching URL https://en.wikipedia.org/wiki/The_Verge (1/535)
Fetching URL https://en.wikipedia.org/wiki/TechCrunch (2/535)
Fetching URL https://en.wikipedia.org/wiki/Manhattan (3/535)
Fetching URL https://en.wikipedia.org/wiki/Fortune (4/535)
Fetching URL https://en.wikipedia.org/wiki/New_York_City (5/535)
Fetching URL https://en.wikipedia.org/wiki/The_Age (6/535)
Fetching URL https://en.wikipedia.org/wiki/generative_AI (7/535)
Fetching URL https://en.wikipedia.org/wiki/OpenAI (8/535)
Fetching URL https://en.wikipedia.org/wiki/ChatGPT (9/535)
Fetching URL https://en.wikipedia.org/wiki/GPT-4 (10/535)
Fetching URL https://en.wikipedia.org/wiki/company (11/535)
Fetching URL https://en.wikipedia.org/wiki/'The_Sydney_Morning_Herald' (12/535)
Fetching URL https://en.wikipedia.org/wiki/'Cnbc_|_World_Business_News_Leader' (13/535)
Fetching URL https://en.wikipedia.org/wiki/Sporting_News (14/535)
Fetching URL https://en.wikipedia.org/wiki/Eagles (15/535)
Fetching URL https://en.wikipedia.org/wi

In [ ]:
# 두 개의 텍스트 파일 내용을 결합하여 새로운 파일에 저장하는 함수
def combine_text_files(file1, file2, output_file):
    # 첫 번째 파일의 내용 읽기
    with open(file1, 'r') as f1:
        content1 = f1.read()

    # 두 번째 파일의 내용 읽기
    with open(file2, 'r') as f2:
        content2 = f2.read()

    # 새 파일에 결합된 내용 쓰기
    with open(output_file, 'w') as out_file:  # 'w' 모드로 새 파일 열기
        out_file.write(content1)
        out_file.write("\n")  # 두 파일 사이에 줄 바꿈 추가
        out_file.write(content2)

    print(f"Contents from {file1} and {file2} combined into {output_file}")

# 사용 예시
#file1 = '/content/drive/MyDrive/QA/MultihopRAG/multihopRAG_corpus.txt'  # 첫 번째 파일 경로
#file2 = '/content/drive/MyDrive/QA/multihopRAG_corpus_wiki.txt'  # 두 번째 파일 경로
#output_file = '/content/drive/MyDrive/QA/MultihopRAG/MultihopRAG_corpus_v2.txt'  # 결합된 내용을 저장할 파일 경로
file1 = '/content/multihopRAG_corpus.txt'  # 첫 번째 파일 경로
file2 = '/content/multihopRAG_corpus_wiki.txt'  # 두 번째 파일 경로
output_file = '/content/MultihopRAG_corpus_500_v3.txt'  # 결합된 내용을 저장할 파일 경로
combine_text_files(file1, file2, output_file)


Contents from /content/multihopRAG_corpus.txt and /content/multihopRAG_corpus_wiki.txt combined into /content/MultihopRAG_corpus_500_v3.txt


In [ ]:
def remove_empty_and_marker_lines(input_file, output_file):
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    # 빈 줄과 '==='로 시작하는 줄을 제외하고 새로운 리스트에 저장
    cleaned_lines = [line for line in lines if line.strip() and not line.startswith("===")]

    # 정리된 내용을 새로운 파일에 저장
    with open(output_file, 'w') as outfile:
        outfile.writelines(cleaned_lines)

    print(f"Cleaned text saved to {output_file}")

# 사용 예시
input_file = '/content/drive/MyDrive/QA/MultihopRAG/MultihopRAG_corpus_v2.txt'  # 원본 텍스트 파일 경로
output_file = '/content/drive/MyDrive/QA/MultihopRAG/MultihopRAG_corpus_v3.txt'  # 정리된 텍스트 파일 경로
remove_empty_and_marker_lines(input_file, output_file)


Cleaned text saved to /content/drive/MyDrive/QA/MultihopRAG/MultihopRAG_corpus_v3.txt
